# Práctica 3 - Redes Neuronales Residuales 

# CARRERA DE MIERDA 

### Natalia Martínez García, Lucía Vega Navarrete
### Grupo: AP.11.06

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, metrics, optimizers, losses
from pathlib import Path
import os
import random

In [2]:
# Fijamos la semilla para poder reproducir los resultados
seed=1234
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

### 1. Carga y preprocesado del dataset 

#### Carga 

In [3]:
DATA_PATH = Path("HIGGS.csv.gz") # El archivo está en la misma carpeta que el script

# Cargar el dataset 
higgs = pd.read_csv(DATA_PATH, compression="gzip", header=None)

print("Tamaño total del dataset (ejemplos, columnas):", higgs.shape)

# Separar características (X) y etiquetas (y)
#   - Columna 0: label
#   - Columnas 1-28: features
X = higgs.iloc[:, 1:].to_numpy(dtype=np.float32) 
y = higgs.iloc[:, 0].to_numpy(dtype=np.float32)

# División según especificaciones: train inicial, test final, extra intermedio
N_TRAIN = 2000000
N_TEST = 500000

X_train = X[:N_TRAIN]
y_train = y[:N_TRAIN]

X_test  = X[-N_TEST:]
y_test  = y[-N_TEST:]

X_extra = X[N_TRAIN:-N_TEST]
y_extra = y[N_TRAIN:-N_TEST]

print("División del dataset:")
print(f"Train: {X_train.shape}")
print(f"Test:  {X_test.shape}")
print(f"Extra: {X_extra.shape}")

Tamaño total del dataset (ejemplos, columnas): (11000000, 29)
División del dataset:
Train: (2000000, 28)
Test:  (500000, 28)
Extra: (8500000, 28)


#### Preprocesado

In [4]:
# Normalizar usando solo TRAIN
# Media y desviación por columna (feature) 
#    (x - media_train) / std_train

"""
Normalizamos solo en train porque cualquier información del test no puede influir en el entrenamiento (el modelo solo puede ver estadísticas del train).
Si usamos la media y desviación del test para normalizar, meteríamos información del futuro dentro del entrenamiento.

Eso se llama data leakage (filtración de información) y hace que tus resultados no sean realistas.
"""
mean_train = X_train.mean(axis=0) # axis = 0 para calcularlo por columnas (cada feature)
std_train  = X_train.std(axis=0)

# Evitar división por cero
std_train[std_train == 0] = 1.0

# Aplicar la transformación estándar: (x - media) / sd
X_train = (X_train - mean_train) / std_train
X_extra = (X_extra - mean_train) / std_train
X_test  = (X_test  - mean_train) / std_train

print("\n Normalización completada")
print(f"- Media global train (ya normalizado): {X_train.mean():.5f}")
print(f"- Std global train  (ya normalizado): {X_train.std():.5f}")

# Comprobación del desbalanceo de clases
def class_stats(name, y):
    n = len(y)
    n_signal = np.sum(y == 1)
    n_back   = np.sum(y == 0)
    print(f"\n{name}:")
    print(f"Signal = {n_signal} ({100*n_signal/n:.2f}%)")
    print(f"Background = {n_back}   ({100*n_back/n:.2f}%)")

print("\n Distribución de clases:")
class_stats("Train", y_train)
class_stats("Extra", y_extra)
class_stats("Test",  y_test)



 Normalización completada
- Media global train (ya normalizado): -0.00000
- Std global train  (ya normalizado): 1.00000

 Distribución de clases:

Train:
Signal = 1058818 (52.94%)
Background = 941182   (47.06%)

Extra:
Signal = 4505798 (53.01%)
Background = 3994202   (46.99%)

Test:
Signal = 264507 (52.90%)
Background = 235493   (47.10%)


### 2. Creación de red neuronal con capas residuales

In [5]:
class ResidualBlock(tf.keras.models.Model):
    """
    Bloque residual con:
      - 2 capas Dense internas
      - 1 conexión residual (skip connection)
    """
    def __init__(self, units, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = activation
    
    def build(self, input_shape):
        """Crea las capas cuando se conoce input_shape"""
        self.dense1 = layers.Dense(self.units, activation=self.activation, name=f"{self.name}_dense1")
        self.dense2 = layers.Dense(self.units, activation=self.activation, name=f"{self.name}_dense2")
        self.add = layers.Add(name=f"{self.name}_add")
        super().build(input_shape)
    
    def call(self, inputs):
        x_skip = inputs
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.add([x, x_skip])
    
    def get_config(self):
        config = super().get_config()
        config.update({"units": self.units, "activation": self.activation})
        return config

In [6]:
class HiggsResNet(keras.Model):
    """
    Red neuronal residual para clasificación del dataset Higgs.
    Arquitectura:
      - Capa de entrada (proyección a dimensión oculta)
      - 2 bloques residuales
      - Capa intermedia
      - Capa de salida (sigmoid para clasificación binaria)
    """
    def __init__(self, input_dim=28, hidden_dim=128, intermediate_dim=64, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.intermediate_dim = intermediate_dim
        
        # Definir todas las capas
        self.entrada = layers.Dense(hidden_dim, activation="relu", name="capa_entrada")
        self.res_block1 = ResidualBlock(hidden_dim, name="res_block1")
        self.res_block2 = ResidualBlock(hidden_dim, name="res_block2")
        self.intermedia = layers.Dense(intermediate_dim, activation="relu", name="capa_intermedia")
        self.salida = layers.Dense(1, activation="sigmoid", name="capa_salida")
    
    def call(self, inputs, training=None):
        """Forward pass del modelo"""
        x = self.entrada(inputs)
        x = self.res_block1(x, training=training)
        x = self.res_block2(x, training=training)
        x = self.intermedia(x)
        outputs = self.salida(x)
        return outputs
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "input_dim": self.input_dim,
            "hidden_dim": self.hidden_dim,
            "intermediate_dim": self.intermediate_dim
        })
        return config

In [7]:
input_dim = X_train.shape[1]  
print("Tamaño de entrada:", input_dim)

# Crear el modelo usando la clase
model = HiggsResNet(input_dim=input_dim, hidden_dim=128, intermediate_dim=64, name="higgs_resnet")

# Construir el modelo llamándolo con datos de ejemplo
_ = model(X_train[:1])

model.summary()

Tamaño de entrada: 28


Model: "higgs_resnet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ capa_entrada (Dense)            │ (1, 128)               │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block1 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block2 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_intermedia (Dense)         │ (1, 64)                │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_salida (Dense)             │ (1, 1)                 │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,081 (305.00 KB)

 Trainable params: 78,081 (305.00 KB)

 Non-trainable params: 0 (0.00 B)

### 3. Entrenamiento de la red

En esta práctica hemos implementado el entrenamiento de la red siguiendo la estructura del tutorial “Escribir un ciclo de entrenamiento desde cero” de TensorFlow, proporcionado en el enunciado de esta práctica.
El código hace exactamente los mismos bloques lógicos propuestos en el tutorial, lo único diferente es que o ampliamos un poco para el cálculo de las métricas F1Score y balanced accurcay. 

1. Preparación de los datos

In [8]:
BATCH_SIZE = 4096

# Dataset de entrenamiento: barajado + dividido en batches
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=100_000, reshuffle_each_iteration=True).batch(BATCH_SIZE)

# Dataset de test: solo en batches, sin shuffle
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_dataset = val_dataset.batch(BATCH_SIZE)

print(train_dataset)
print(val_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


2. Definir loss, optimizer y métricas

In [9]:
optimizer = optimizers.Adam(learning_rate=1e-3) # Optimizador
loss_fn = losses.BinaryCrossentropy() # Función de pérdida (binaria, porque es 0/1)


# Métricas para TRAIN
train_loss = metrics.Mean(name="train_loss")
train_accuracy = metrics.BinaryAccuracy(name="train_accuracy")

# Métricas para TEST
test_loss = metrics.Mean(name="test_loss")
test_accuracy = metrics.BinaryAccuracy(name="test_accuracy")
# Toca calcular el F1 a mano tb xq me estaba dando un error de dimensionalidad por usar la clase q lo hace directo de keras lmao 
test_precision = metrics.Precision(name="test_precision")
test_recall    = metrics.Recall(name="test_recall")

3. Esto está copiado tal cual el tutorial, se supone q hace el entrenamiento más rápido 

`@tf.function` convierte las funciones en grafos de TensorFlow optimizados, lo que hace que el entrenamiento sea más rápido y eficiente.

`train_step(x, y)` realiza un paso completo de entrenamiento (forward, cálculo de loss, gradientes, actualización de pesos y de métricas).

`test_step(x, y)` realiza la evaluación sobre un batch sin actualizar los pesos, solo actualizando las métricas de validación.

In [10]:
@tf.function
def train_step(x, y):
    """Paso de entrenamiento: forward + loss + backprop + métricas train."""
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss_value = loss_fn(y, predictions)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # Actualizar métricas de entrenamiento
    train_loss.update_state(loss_value)
    train_accuracy.update_state(y, predictions)
    return loss_value
    
@tf.function
def test_step(x, y):
    """
    Paso de evaluación:
      - forward
      - actualizar métricas de test (loss, accuracy, precision, recall)
    Devuelve las predicciones para poder calcular balanced accuracy fuera.
    """
    predictions = model(x, training=False)
    loss_value = loss_fn(y, predictions)

    test_loss.update_state(loss_value)
    test_accuracy.update_state(y, predictions)
    test_precision.update_state(y, predictions)
    test_recall.update_state(y, predictions)

    return predictions

4. Bucle de epochs + F1 score + balanced accuracy + guardado 

La estructura es la misma que el tutorial:
* Loop de validación:
    * for x_batch_val, y_batch_val in test_ds: val_predictions = test_step(...)
* Leer métricas .result().
* Resetear métricas (reset_state()).
* Imprimir métricas + tiempo.

In [11]:
import time

EPOCHS = 3 

for epoch in range(EPOCHS):
    print("\nSTART OF EPOCH %d" % (epoch + 1,))
    print("")
    start_time = time.time()

    # FASE DE ENTRENAMIENTO
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log cada 200 batches 
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * BATCH_SIZE))

    # Mostrar las métricas al final de cada epoch 
    train_acc  = float(train_accuracy.result())
    train_loss_epoch = float(train_loss.result())
    print("\n--- Training metrics ---")
    print("Accuracy: %.4f" % train_acc)
    print("Loss: %.4f" % train_loss_epoch)

    # Reset training metrics at the end of each epoch
    train_loss.reset_state()
    train_accuracy.reset_state()

    # FASE DE EVALUACIÓN (TEST)
    # Esto lo usamos luego para el cálculo de balances accuracy 
    y_true_all = [] # todas las estiquetas reales 
    y_pred_all = [] # todas las predicciones binarias 

    for x_batch_val, y_batch_val in val_dataset:
        # test_step actualiza loss, accuracy, precision, recall
        val_predictions = test_step(x_batch_val, y_batch_val)

        # Guardar para balanced accuracy
        y_true_all.append(y_batch_val.numpy().astype(np.int32))
        y_pred_all.append((val_predictions.numpy() >= 0.5).astype(np.int32))

    y_true_all = np.concatenate(y_true_all, axis=0).reshape(-1)
    y_pred_all = np.concatenate(y_pred_all, axis=0).reshape(-1)

    # Métricas de test a partir de los objetos de Keras 
    val_loss = float(test_loss.result())
    val_acc = float(test_accuracy.result())
    precision = float(test_precision.result())
    recall = float(test_recall.result())
    f1_score = 2 * precision * recall / (precision + recall + 1e-8)

    # Reset de métricas de test para la siguiente época
    test_loss.reset_state()
    test_accuracy.reset_state()
    test_precision.reset_state()
    test_recall.reset_state()

    # Balanced Accuracy (a mano)
    # identificamos los índices donde las etiquetas son 0/1
    mask_0 = (y_true_all == 0) # pone a True todos los valores de la lista de etiquetas reales que valgan 0
    mask_1 = (y_true_all == 1) # pone a True todos los valores de la lista de etiquetas reales que valgan 1

    acc_class0 = np.mean(y_pred_all[mask_0] == y_true_all[mask_0])
    acc_class1 = np.mean(y_pred_all[mask_1] == y_true_all[mask_1])
    balanced_accuracy = (acc_class0 + acc_class1) / 2

    # Log de validación como en el tutorial, pero ampliado 
    print("\n--- Validation metrics ---")
    print("Loss: %.4f" % val_loss)
    print("Accuracy: %.4f" % val_acc)
    print("Precision: %.4f" % precision)
    print("Recall: %.4f" % recall)
    print("F1Score: %.4f" % f1_score)

    print("\n--- Balanced Accuracy ---")
    print("Class 0 (background): %.4f" % acc_class0)
    print("Class 1 (signal): %.4f" % acc_class1)
    print("Media: %.4f" % balanced_accuracy)

    print("")
    print("Time taken: %.2fs" % (time.time() - start_time))
    print("-"*50)

# Guardar el modelo entrenado
MODEL_PATH = "higgs_resnet_trained.keras"
model.save(MODEL_PATH)
print(f"\nModelo guardado en: {MODEL_PATH}")



START OF EPOCH 1

Training loss (for one batch) at step 0: 0.8607
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.5656
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.5359
Seen so far: 1642496 samples

--- Training metrics ---
Accuracy: 0.6940
Loss: 0.5745

--- Validation metrics ---
Loss: 0.5409
Accuracy: 0.7241
Precision: 0.7163
Recall: 0.7923
F1Score: 0.7524

--- Balanced Accuracy ---
Class 0 (background): 0.6476
Class 1 (signal): 0.7923
Media: 0.7199

Time taken: 8.28s
--------------------------------------------------

START OF EPOCH 2

Training loss (for one batch) at step 0: 0.5365
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.5389
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.5326
Seen so far: 1642496 samples

--- Training metrics ---
Accuracy: 0.7331
Loss: 0.5269

--- Validation metrics ---
Loss: 0.5169
Accuracy: 0.7390
Precision: 0.7478
Recall: 0.7643
F1Score: 0.7560

--- Bala

FINE TUNING

In [ ]:
# Cargar el modelo entrenado
model_base = keras.models.load_model(
    "higgs_resnet_trained.keras",
    custom_objects={'ResidualBlock': ResidualBlock, 'HiggsResNet': HiggsResNet}
)

# Congelar TODAS las capas
for layer in model_base.layers:
    layer.trainable = False

# Verificar que están congeladas (NO SE SI DEJAR ESTO ??????)
print("Capas congeladas:")
for layer in model_base.layers:
    print(f"  - {layer.name}: trainable = {layer.trainable}")

In [ ]:
# AÑADIR MÓDULOS DE ADAPTACIÓN DE BAJA DIMENSIÓN 
# CLASE PORQUE VAMOS A REPETIR ESTO EN CADA CAPA ????????????

class LoRADense(layers.Layer):
    """
    Capa Dense con adaptación LoRA (Low-Rank Adaptation).
    
    Funciona así:
      y = (W + α*B*A) * x + b
    
    Donde:
      - W son los pesos originales (congelados)
      - A y B son matrices pequeñas (entrenables)
      - α controla cuánta influencia tiene la adaptación
    """
    def __init__(self, dense_layer, rank=4, alpha=0.1, **kwargs):
        super().__init__(**kwargs)
        self.dense_layer = dense_layer
        self.rank = rank  # r en la fórmula
        self.alpha = alpha  # α en la fórmula
        
        # Congelar la capa densa original ( ESTO ES REDUNDANTE ??????)
        self.dense_layer.trainable = False
        
        # Dimensiones de los pesos originales W
        self.units = dense_layer.units  # dout
        self.input_dim = None  # din (se determina en build)
        
    def build(self, input_shape):
        self.input_dim = input_shape[-1]  # din
        
        # Crear matrices A y B de baja dimensión
        # A: (r × din) - inicializada con valores aleatorios
        self.lora_A = self.add_weight(
            name='lora_A',
            shape=(self.rank, self.input_dim),
            # NOTA: MIRAR SI ESTO DE VERDAD ES INICIALIZACION ALEATORIA
            initializer='glorot_uniform',  # Valores aleatorios
            trainable=True
        )
        
        # B: (dout × r) - inicializada a CERO
        self.lora_B = self.add_weight(
            name='lora_B',
            shape=(self.units, self.rank),
            initializer='zeros',  # Todo a cero
            trainable=True
        )
        
        super().build(input_shape)
    
    def call(self, inputs):
        # Salida de la capa original (W*x + b)
        base_output = self.dense_layer(inputs)
        
        # Calcular la adaptación: α * B * A * x
        # Paso 1: A * x  → resultado tiene forma (batch, r)
        # tf.matmul es una función de TensorFlow que multiplica dos matrices
        lora_output = tf.matmul(inputs, self.lora_A, transpose_b=True)
        
        # Paso 2: B * (A * x)  → resultado tiene forma (batch, dout)
        lora_output = tf.matmul(lora_output, self.lora_B, transpose_b=True)
        
        # Aplicar el factor α
        lora_output = self.alpha * lora_output
        
        # Combinar: (W*x + b) + α*B*A*x
        return base_output + lora_output
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'rank': self.rank,
            'alpha': self.alpha
        })
        return config

In [ ]:
# ESTO CAMBIARLO FIJO. POR QUE HACER UNA CLASE NUEVA ??????'

# Función para crear modelo con LoRA
def create_lora_model(base_model, rank=4, alpha=0.1):
    """
    Envuelve todas las capas Dense del modelo base con LoRA.
    """
    # Envolver cada capa Dense con LoRA
    lora_entrada = LoRADense(base_model.entrada, rank=rank, alpha=alpha, name="lora_entrada")
    
    lora_res1_d1 = LoRADense(base_model.res_block1.dense1, rank=rank, alpha=alpha, name="lora_res1_dense1")
    lora_res1_d2 = LoRADense(base_model.res_block1.dense2, rank=rank, alpha=alpha, name="lora_res1_dense2")
    
    lora_res2_d1 = LoRADense(base_model.res_block2.dense1, rank=rank, alpha=alpha, name="lora_res2_dense1")
    lora_res2_d2 = LoRADense(base_model.res_block2.dense2, rank=rank, alpha=alpha, name="lora_res2_dense2")
    
    lora_intermedia = LoRADense(base_model.intermedia, rank=rank, alpha=alpha, name="lora_intermedia")
    lora_salida = LoRADense(base_model.salida, rank=rank, alpha=alpha, name="lora_salida")
    
    # Crear el modelo adaptado
    class HiggsResNetLoRA(keras.Model):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.lora_entrada = lora_entrada
            self.lora_res1_d1 = lora_res1_d1
            self.lora_res1_d2 = lora_res1_d2
            self.lora_res2_d1 = lora_res2_d1
            self.lora_res2_d2 = lora_res2_d2
            self.lora_intermedia = lora_intermedia
            self.lora_salida = lora_salida
            self.add_layer = layers.Add()
        
        def call(self, inputs, training=None):
            # Capa de entrada
            x = self.lora_entrada(inputs)
            
            # Bloque residual 1
            x_skip = x
            x = self.lora_res1_d1(x)
            x = self.lora_res1_d2(x)
            x = self.add_layer([x, x_skip])
            
            # Bloque residual 2
            x_skip = x
            x = self.lora_res2_d1(x)
            x = self.lora_res2_d2(x)
            x = self.add_layer([x, x_skip])
            
            # Capas finales
            x = self.lora_intermedia(x)
            outputs = self.lora_salida(x)
            
            return outputs
    
    return HiggsResNetLoRA()


# Crear el modelo con LoRA
RANK = 8  # Valor pequeño (prueba con 4, 8, 16)
ALPHA = 0.1  # Factor de escala (0.1 funciona bien)

model_lora = create_lora_model(model_base, rank=RANK, alpha=ALPHA)

# Construir el modelo
_ = model_lora(X_train[:1])

# Contar parámetros
trainable_params = sum([tf.size(w).numpy() for w in model_lora.trainable_weights])
total_params = sum([tf.size(w).numpy() for w in model_lora.weights])

print(f"\nParámetros del modelo LoRA:")
print(f"  - Total: {total_params:,}")
print(f"  - Entrenables (solo LoRA): {trainable_params:,}")
print(f"  - Congelados (modelo base): {total_params - trainable_params:,}")
print(f"  - Porcentaje entrenable: {100 * trainable_params / total_params:.2f}%")

In [ ]:
# MIRAR SI SE PUEDE ENTRENAR CON FIT O NO 

# ENTRENAR EL MODELO ADAPTADO CON LOS DATOS EXTRA
print("\n=== PASO 4c: Entrenando modelo LoRA ===\n")

# Preparar datos extra
extra_dataset = tf.data.Dataset.from_tensor_slices((X_extra, y_extra))
extra_dataset = extra_dataset.shuffle(buffer_size=100_000).batch(BATCH_SIZE)

# Compilar modelo LoRA
model_lora.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss=losses.BinaryCrossentropy(),
    metrics=[
        metrics.BinaryAccuracy(name='accuracy'),
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall')
    ]
)

# Entrenar (menos épocas porque solo ajustamos parámetros pequeños)
EPOCHS_LORA = 2

history_lora = model_lora.fit(
    extra_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS_LORA,
    verbose=1
)

print("\nModelo LoRA entrenado correctamente")

In [ ]:
# 4d. EVALUAR Y COMPARAR RESULTADOS
print("\n=== PASO 4d: Evaluación y comparación ===\n")

def evaluate_model(model, X, y, model_name):
    """Evalúa un modelo y calcula todas las métricas"""
    predictions = model.predict(X, verbose=0)
    y_pred_binary = (predictions >= 0.5).astype(np.int32).reshape(-1)
    y_true = y.astype(np.int32)
    
    # Accuracy
    accuracy = np.mean(y_pred_binary == y_true)
    
    # Precision y Recall
    tp = np.sum((y_pred_binary == 1) & (y_true == 1))
    fp = np.sum((y_pred_binary == 1) & (y_true == 0))
    fn = np.sum((y_pred_binary == 0) & (y_true == 1))
    
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1_score = 2 * precision * recall / (precision + recall + 1e-8)
    
    # Balanced Accuracy
    mask_0 = (y_true == 0)
    mask_1 = (y_true == 1)
    acc_class0 = np.mean(y_pred_binary[mask_0] == y_true[mask_0])
    acc_class1 = np.mean(y_pred_binary[mask_1] == y_true[mask_1])
    balanced_acc = (acc_class0 + acc_class1) / 2
    
    print(f"\n{model_name}:")
    print(f"  Accuracy:          {accuracy:.4f}")
    print(f"  Balanced Accuracy: {balanced_acc:.4f}")
    print(f"  Precision:         {precision:.4f}")
    print(f"  Recall:            {recall:.4f}")
    print(f"  F1-Score:          {f1_score:.4f}")
    
    return {
        'accuracy': accuracy,
        'balanced_accuracy': balanced_acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

# Evaluar modelo base (sin adaptación)
results_base = evaluate_model(model_base, X_test, y_test, "MODELO BASE (sin fine-tuning)")

# Evaluar modelo con LoRA
results_lora = evaluate_model(model_lora, X_test, y_test, "MODELO CON LoRA (con fine-tuning)")

# Comparación
print("\n" + "="*60)
print("COMPARACIÓN DE MEJORAS")
print("="*60)
for metric in ['accuracy', 'balanced_accuracy', 'f1_score']:
    improvement = results_lora[metric] - results_base[metric]
    print(f"{metric.replace('_', ' ').title():20s}: {improvement:+.4f}")

In [ ]:
# 4e. COMPACTAR W, A y B EN UNA ÚNICA MATRIZ
print("\n=== PASO 4e: Compactando LoRA en pesos originales ===\n")

def merge_lora_weights(model_lora):
    """
    Combina W + α*B*A en una única matriz para cada capa.
    Esto hace que el modelo sea igual de rápido que el original.
    """
    print("Fusionando pesos LoRA con pesos base...")
    
    # Lista de capas LoRA
    lora_layers = [
        model_lora.lora_entrada,
        model_lora.lora_res1_d1,
        model_lora.lora_res1_d2,
        model_lora.lora_res2_d1,
        model_lora.lora_res2_d2,
        model_lora.lora_intermedia,
        model_lora.lora_salida
    ]
    
    for lora_layer in lora_layers:
        # Obtener pesos originales W
        original_weights = lora_layer.dense_layer.get_weights()
        W = original_weights[0]  # Matriz de pesos
        b = original_weights[1]  # Bias
        
        # Obtener matrices LoRA
        A = lora_layer.lora_A.numpy()  # (r × din)
        B = lora_layer.lora_B.numpy()  # (dout × r)
        alpha = lora_layer.alpha
        
        # Calcular: W_new = W + α*B*A
        # B*A resulta en una matriz (dout × din), igual que W
        BA = np.matmul(B, A)  # (dout × r) @ (r × din) = (dout × din)
        W_new = W + alpha * BA.T  # Transponer porque TensorFlow usa (din × dout)
        
        # Actualizar pesos de la capa original
        lora_layer.dense_layer.set_weights([W_new, b])
        
        print(f"  ✓ {lora_layer.name}: fusionado")
    
    print("\nTodos los pesos LoRA han sido fusionados con los pesos base")
    return model_lora

# Fusionar pesos
model_merged = merge_lora_weights(model_lora)

# Verificar que da el mismo resultado
print("\nVerificando que el modelo fusionado da los mismos resultados...")
pred_lora = model_lora.predict(X_test[:1000], verbose=0)
pred_merged = model_merged.predict(X_test[:1000], verbose=0)

difference = np.abs(pred_lora - pred_merged).max()
print(f"Diferencia máxima entre predicciones: {difference:.10f}")

if difference < 1e-5:
    print("✓ Las predicciones son idénticas (diferencia despreciable)")
else:
    print("⚠ Hay diferencias significativas")

# Evaluar modelo fusionado
results_merged = evaluate_model(model_merged, X_test, y_test, "MODELO FUSIONADO (W + α*B*A)")

# Guardar modelo final
model_merged.save("higgs_resnet_finetuned.keras")
print("\nModelo fusionado guardado en: higgs_resnet_finetuned.keras")